In [137]:
import numpy as np
import pandas as pd
from pprint import pprint

In [101]:
df_train = pd.read_csv("HW1/all_data/train_c300_d100.csv", header=None)

In [141]:
df_train.head()
a={"a":1, "b": 3, "c": 2}
max_key = max(a, key=a.get)
print(max_key)

b


In [94]:
# df_train = df.iloc[:,:30]
# df_label = df.iloc[:,-1]
# print(df_label.head())
# print(df_train.shape)
# df_final = df_train.join(df_label)
# print(df_final.shape)
# print(df_train.shape)

In [95]:
data = np.array([1])
if len(data) == 1:
    print(0)

0


In [96]:
if not 0 in data:
    print("data has 0")

data has 0


In [97]:
def check_purity(data):
    
    label_column = data[:, -1]
    unique_classes = np.unique(label_column)
    
    if len(unique_classes) == 1:
        return True
    else:
        return False

In [98]:
def classify_data(data):
    label = data[:,-1]
    classes, counts = np.unique(label, return_counts=True)
#     print(classes, counts)
    classification = classes[counts.argmax()]
    return classification

In [99]:
check_purity(df_train[1:2].values)

True

In [102]:
range(len(df_train.columns) - 1)

range(0, 500)

## Split Data

In [103]:
def get_potential_splits(data):
    _, n_columns = data.shape
    
    #last column  is label and is not considered
    return range(n_columns - 1)    

In [104]:
get_potential_splits(df_train.values)

range(0, 500)

In [105]:
def split_data(data, split_column):
    split_column_val = data[:, split_column]
    data_attr_false = data[split_column_val == 0]
    data_attr_true = data[split_column_val == 1]
    return data_attr_false, data_attr_true

In [106]:
split_column = 9
data_F, data_T = split_data(df_train.values, split_column)
print(data_F[:, -1], data_T[:, -1])
print(data_F.shape, data_T.shape)

[1 0 1 1 1 0 0 0 0 1 1 1 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 1 0 1 0 1 0 1 0 0
 1 0 1 1 0 1 1 0 0 1 1 0 1 1 0 0 1 1 0 0 0 1 1 1 1 1 1 0 0 0 1 1 0 1 0 0 0
 0 0 0 0 0] [0 1 0 0 1 1 1 1 0 1 0 1 0 1 0 0 0 1 0 1 0 1 0 1 1 0 1 0 1 1 0 1 0 1 1 0 1
 0 0 0 0 0 1 0 1 0 0 1 0 1 0 0 1 0 1 0 1 0 0 1 0 1 1 0 1 1 0 0 1 0 0 1 0 1
 0 0 0 1 0 1 0 1 1 1 0 0 0 1 1 1 0 1 0 0 0 0 0 1 0 1 0 1 1 1 0 0 0 1 0 1 0
 1 1 1 1 1 0 1 1 1 1]
(79, 501) (121, 501)


In [107]:
def calculate_entropy(data):
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)
    probabilities = counts / counts.sum()
#     print(probabilities, counts)
    entropy = sum(probabilities * -np.log2(probabilities))
    return entropy

In [108]:
calculate_entropy(df_train.values)

1.0

In [109]:
def determine_best_split(data, potential_splits):
    best_info_gain = None
    for column_index in potential_splits:
        data_f, data_t = split_data(data, column_index)
        
        #Info Gain
        info_gain = ( calculate_entropy(data) - 
                     ((len(data_f)/len(data)) * calculate_entropy(data_f)) - 
                     ((len(data_t)/len(data)) * calculate_entropy(data_t)) )
        
        if best_info_gain is None:
            best_info_gain = info_gain
            best_split_column = column_index
        elif info_gain >= best_info_gain:
            best_info_gain = info_gain
            best_split_column = column_index
#             print(column_index, info_gain)
    return best_split_column

In [110]:
split_col = determine_best_split(df_train.values, get_potential_splits(df_train.values))
print(split_col)

459


## Main Algorithm

In [134]:
def decision_tree_algorithm(data, depth=0, dmax=5):
    
    # base case
    if ( check_purity(data) == True or
        depth >= dmax ):
        classification = classify_data(data)
        return classification
    
    #recursion
    else:
        depth+=1
        
        split_column = determine_best_split(data, get_potential_splits(data))
        data_0, data_1 = split_data(data, split_column)
        
        if len(data_0) == 0 or len(data_1) == 0:
            classification = classify_data(data)
            return classification
        
        question = f"{split_column} == 0"
        
        #instantiate sub-tree
        sub_tree = {question: []}
        
        is_0 = decision_tree_algorithm(data_0, depth)
        is_1 = decision_tree_algorithm(data_1, depth)
        
        sub_tree[question].append(is_0)
        sub_tree[question].append(is_1)
        
        return sub_tree

In [138]:
tree = decision_tree_algorithm(df_train.values)
pprint(tree)

{'459 == 0': [{'430 == 0': [{'118 == 0': [{'389 == 0': [{'468 == 0': [0, 1]},
                                                        0]},
                                          {'445 == 0': [{'356 == 0': [1, 0]},
                                                        {'351 == 0': [1,
                                                                      0]}]}]},
                            {'306 == 0': [{'234 == 0': [{'379 == 0': [0, 1]},
                                                        {'312 == 0': [0, 1]}]},
                                          {'172 == 0': [{'360 == 0': [0, 1]},
                                                        1]}]}]},
              {'321 == 0': [{'311 == 0': [{'383 == 0': [1,
                                                        {'498 == 0': [1, 0]}]},
                                          {'434 == 0': [{'101 == 0': [0, 1]},
                                                        0]}]},
                            {'475 == 0': [{'144 ==

## Classification

In [113]:
df_test = pd.read_csv("HW1/all_data/test_c300_d100.csv", header=None)
# df_test_data = df.iloc[:, :30]
# df_test_label = df.iloc[:, -1]
# df_test = df_test_data.join(df_test_label)
example = df_test.iloc[0]
print(example)

0      1
1      0
2      0
3      1
4      0
      ..
496    1
497    1
498    0
499    0
500    1
Name: 0, Length: 501, dtype: int64


In [114]:
def classify_example(example, tree, print_path=0):
    question = list(tree.keys())[0]
    if print_path != 0:
        print(question)
    feature_index,_, _ = question.split()
    
    if example[int(feature_index)] == 0:
        answer = tree[question][0]
        if print_path!=0:
            print("yes")
    else:
        answer = tree[question][1]
        if print_path!=0:
            print("no")
            
    # base case
    if not isinstance(answer, dict):
        return answer
    else:
        return classify_example(example, answer, print_path)

In [115]:
answer = classify_example(example, tree, 1)
print(answer)
classification = df_test.apply(classify_example, axis=1, args=(tree, ))
print((classification == df_test[df_test.columns[-1]]).mean())

459 == 0
yes
430 == 0
yes
118 == 0
no
445 == 0
no
351 == 0
yes
1
0.615


## Accuracy

In [119]:
def calculate_accuracy(df, tree):
    classification = df.apply(classify_example, axis=1, args=(tree, ))
#     print(classification)
#     print(df[df.columns[-1]])
    classification_correct = classification == df[df.columns[-1]]
#     print(classification_correct)
    accuracy = classification_correct.mean()
    return accuracy

In [120]:
# print(df_final)
calculate_accuracy(df_test, tree)

0.615

## Post_pruning

In [121]:
df_valid = pd.read_csv("HW1/all_data/valid_c300_d100.csv", header=None)
# df_valid_data = df.iloc[:, :30]
# df_valid_label = df.iloc[:, -1]
# df_valid = df_valid_data.join(df_valid_label)
# print(df_valid_label)
# print(df_valid[df_valid.columns[-1]])

In [122]:
def filter_df(df, question):
    index, _, _ = question.split()
    df_f = df[df[int(index)] == 0]
    df_t = df[df[int(index)] != 0]
    
    return df_f, df_t

In [123]:
df_f, df_t = filter_df(df_valid, "0 == 0")
print(f"false: {df_f},true: {df_t}")

false:      0    1    2    3    4    5    6    7    8    9    ...  491  492  493  \
1      0    1    1    0    0    1    1    1    0    1  ...    1    0    1   
4      0    0    1    0    0    0    0    0    1    0  ...    1    0    0   
5      0    0    1    1    0    0    0    1    0    0  ...    0    0    0   
6      0    1    0    1    0    1    0    1    1    0  ...    1    1    0   
9      0    0    0    0    0    1    0    0    0    0  ...    0    0    1   
..   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
190    0    0    0    1    0    1    0    1    1    1  ...    1    0    1   
191    0    0    1    1    0    0    0    0    1    1  ...    1    1    1   
193    0    0    1    0    1    1    0    0    1    1  ...    1    0    1   
195    0    1    0    0    0    1    0    0    0    0  ...    0    0    0   
196    0    1    1    0    0    1    1    0    0    1  ...    1    0    0   

     494  495  496  497  498  499  500  
1      1    1    1    1    

In [124]:
def pruning_result(tree, df_train, df_val):
    leaf = df_train[df_train.columns[-1]].value_counts().index[0]
    errors_leaf = sum(df_val[df_val.columns[-1]] != leaf)
    classification = df_val.apply(classify_example, axis=1, args=(tree, ))
    errors_decision_node = sum(df_val[df_val.columns[-1]] != classification)
    
    if errors_leaf <= errors_decision_node:
        return leaf
    else:
        return tree

In [125]:
final = pruning_result(tree, df_train, df_valid)
print(final)

{'459 == 0': [{'430 == 0': [{'118 == 0': [{'389 == 0': [{'468 == 0': [0, 1]}, 0]}, {'445 == 0': [{'356 == 0': [{'329 == 0': [{'410 == 0': [{'496 == 0': [0, 1]}, 1]}, 0]}, 0]}, {'351 == 0': [1, {'498 == 0': [1, 0]}]}]}]}, {'306 == 0': [{'234 == 0': [{'379 == 0': [{'496 == 0': [0, 1]}, 1]}, {'312 == 0': [0, {'195 == 0': [1, 0]}]}]}, {'172 == 0': [{'360 == 0': [{'497 == 0': [1, 0]}, 1]}, 1]}]}]}, {'321 == 0': [{'311 == 0': [{'383 == 0': [1, {'498 == 0': [1, 0]}]}, {'434 == 0': [{'101 == 0': [0, {'494 == 0': [1, 0]}]}, 0]}]}, {'475 == 0': [{'144 == 0': [0, {'494 == 0': [1, 0]}]}, 0]}]}]}


In [126]:
def post_pruning(tree, df_train, df_val):
    question = list(tree.keys())[0]
    yes_answer, no_answer = tree[question]
    
    # base case
    if not isinstance(yes_answer, dict) and not isinstance(no_answer, dict):
        return pruning_result(tree, df_train, df_val)
    
    else:
        df_train_yes, df_train_no = filter_df(df_train, question)
        df_val_yes, df_val_no = filter_df(df_val, question)
        
        if isinstance(yes_answer, dict):
            yes_answer = post_pruning(yes_answer, df_train_yes, df_val_yes)
        
        if isinstance(no_answer, dict):
            no_answer = post_pruning(no_answer, df_train_no, df_val_no)
        
        tree = {question: [yes_answer, no_answer]}
        
    return pruning_result(tree, df_train, df_val)

In [127]:
pruned_tree = post_pruning(tree, df_train, df_valid)

In [128]:
pruned_tree

{'459 == 0': [{'430 == 0': [{'118 == 0': [0,
      {'445 == 0': [{'356 == 0': [1, 0]}, 1]}]},
    1]},
  {'321 == 0': [{'311 == 0': [1, 0]}, 0]}]}

In [129]:
tree

{'459 == 0': [{'430 == 0': [{'118 == 0': [{'389 == 0': [{'468 == 0': [0, 1]},
        0]},
      {'445 == 0': [{'356 == 0': [{'329 == 0': [{'410 == 0': [{'496 == 0': [0,
                1]},
              1]},
            0]},
          0]},
        {'351 == 0': [1, {'498 == 0': [1, 0]}]}]}]},
    {'306 == 0': [{'234 == 0': [{'379 == 0': [{'496 == 0': [0, 1]}, 1]},
        {'312 == 0': [0, {'195 == 0': [1, 0]}]}]},
      {'172 == 0': [{'360 == 0': [{'497 == 0': [1, 0]}, 1]}, 1]}]}]},
  {'321 == 0': [{'311 == 0': [{'383 == 0': [1, {'498 == 0': [1, 0]}]},
      {'434 == 0': [{'101 == 0': [0, {'494 == 0': [1, 0]}]}, 0]}]},
    {'475 == 0': [{'144 == 0': [0, {'494 == 0': [1, 0]}]}, 0]}]}]}

In [130]:
calculate_accuracy(df_test, pruned_tree)

0.645